### Build a simple LLM Application with LCEL

in the quickest start we will show you how to build a simple llm application with Langchain this application will translate text from English into another language this is a relatively simple LLM aplicacion it suggests a single llm call plus some prompting it's still this is a great way to get started with langchain and a lot of feature can be built with just some prompting an an alarm call

overview - 

- Using language models
- Using PromptTemplates and Outputparser
- using expression Language LCEL To chain component together
- Debugging and tracing your application using language
- deploying your application with LangServe




In [ ]:
### Open AI Key and open source model configuration
import os 
from  dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## for langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT_NAME"] = "LangchainFramework"



In [9]:
import os
from langchain_groq import ChatGroq

ChatGroq.api_key = os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

In [11]:
from langchain_core.messages import SystemMessage, HumanMessage
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="Translate 'Hello, how are you?' to French."),
]

responce = model.invoke(messages)

In [13]:
responce

AIMessage(content='The translation of \'Hello, how are you?\' to French is:\n\n"Bonjour, comment allez-vous?"\n\nThis is a formal way to ask the question. If you want to use an informal tone, you can say:\n\n"Salut, comment vas-tu?"', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 57, 'total_tokens': 112, 'completion_time': 0.164875255, 'completion_tokens_details': None, 'prompt_time': 0.005223005, 'prompt_tokens_details': None, 'queue_time': 0.056658625, 'total_time': 0.17009826}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd0b4-bc26-7203-b354-604f1ea50d11-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 55, 'total_tokens': 112})

In [15]:
## Now we use stroutput parser to get structured output
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()  
parser.invoke(responce)


'The translation of \'Hello, how are you?\' to French is:\n\n"Bonjour, comment allez-vous?"\n\nThis is a formal way to ask the question. If you want to use an informal tone, you can say:\n\n"Salut, comment vas-tu?"'

In [16]:
## Using LCEL - Chain the components together
chain = model|parser
chain.invoke(messages)

'The translation of \'Hello, how are you?\' to French is:\n\n"Bonjour, comment allez-vous?"\n\nThis is a formal way to ask the question. If you want to ask an informal version, you can say:\n\n"Salut, comment vas-tu?"'

In [ ]:
## Create prompt using ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate

template = "Translate the following into: {language}."

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("user", "{text}")
    ]
)


In [20]:
result = prompt.invoke({"language": "French", "text": "Hello, how are you?"})

In [21]:
result.to_messages()

[SystemMessage(content='Translate the following into: French.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [22]:
chain = prompt | model | parser
chain.invoke({"language": "French", "text": "Hello, how are you?"})

'Bonjour, comment allez-vous ?'

In [ ]:
## Rest API is an arcitecture and fast api is python framework which helps to build rest apis quickly and easily.